<a href="https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/Cascading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://withpi.ai"><img src="https://withpi.ai/logoFullBlack.svg" width="240"></a>

<a href="https://code.withpi.ai"><font size="4">Documentation</font></a>

<a href="https://play.withpi.ai"><font size="4">Technique Catalog</font></a>

# WithPi Contract Creation

You have a generative AI application, but you aren't happy with its responses to user prompts.  The **WithPi** SDK helps you build a feedback loop, making your responses get better automatically without significant Quality expertise.  Let's dig in!

This Colab walks you through the first step, creating a **Pi Contract**.  This is a **human and machine readable** description of what **goodness** means to you and is the cornerstone of our approach.

This should take about **15 minutes**, even if you're unfamiliar with Colab.

## Install and initialize SDK

Connect to a regular CPU Python 3 runtime.  You won't need GPUs for this notebook.

You'll need a WITHPI_API_KEY from https://play.withpi.ai.  Add it to your notebook secrets (the key symbol) on the left.

Run the cell below to install packages and load the SDK

In [ ]:
%%capture

%pip install withpi litellm

import os
from google.colab import userdata
from litellm import completion
from withpi import PiClient

os.environ["WITHPI_API_KEY"] = userdata.get('WITHPI_API_KEY')

client = PiClient()

def print_contract(contract):
  for dimension in contract.dimensions:
    print(dimension.label)
    for sub_dimension in dimension.sub_dimensions:
      print(f"\t{sub_dimension.description}")

def generate(system: str, user: str, model: str) -> str:
  messages = [
    {
      "content": system,
      "role": "system"
    },
    {
      "content": prompt,
      "role": "user"
    }
  ]
  return completion(model=model,
                    messages=messages).choices[0].message.content

class printer(str):
  def __repr__(self):
    return self
def print_response(response: str):
  display(printer(response))

def print_scores(pi_scores):
  for dimension_name, dimension_scores in pi_scores.dimension_scores.items():
    print(f"{dimension_name}: {dimension_scores.total_score}")
    for subdimension_name, subdimension_score in dimension_scores.subdimension_scores.items():
      print(f"\t{subdimension_name}: {subdimension_score}")
    print("\n")
  print("---------------------")
  print(f"Total score: {pi_scores.total_score}")

# Make a contract

Let's say you want to build an application that generates children's stories teaching a life lesson.  Call it `AesopAI`.

Start by creating a first cut contract based on that general input, proposed in the following cell:


In [ ]:
aesop_contract = client.contracts.generate_dimensions(
    contract_description=(
        "Write a children's story in the style of Aesop's Fables "
        "teaching a life lesson specified by the user. Provide just the "
        "story with no extra content."
    ),
)

for dimension in aesop_contract.dimensions:
  print(dimension.label)
  for sub_dimension in dimension.sub_dimensions:
    print(f"\t{sub_dimension.description}")

A contract is essentially a hierarchical rubric for grading a response.  A bunch of "simple" questions add up to broader categories, which yield a final score.  Output will vary somewhat, but the table above should have reasonable grading questions for the application.

## Smoke test the contract

Let's see how it performs with no further tuning.  The below cell uses Gemini to generate a response, but any suitable model will work fine.

Adjust to pick a different model and supply your own key with docs at https://docs.litellm.ai/docs/.

You can import a Google Gemini key from AI Studio on the left pane, which populates a GOOGLE_API_KEY secret essentially for free.

In [ ]:
from litellm import completion
import os
from google.colab import userdata

os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')

def generate(prompt: str) -> str:
  messages = [
      {
          "content": aesop_contract.description,
          "role": "system"
      },
      {
          "content": prompt,
          "role": "user"
      },
  ]
  return completion(model="gemini/gemini-2.0-flash-exp",
                    messages=messages).choices[0].message.content

prompt = "The importance of sharing"
response = generate(prompt)

# Print with line wrapping and explicit newlines.
class printer(str):
    def __repr__(self):
       return self
display(printer(response))

## Score it!

Take the generated response and see how it scores with Pi.

The below cell will run Pi Scoring, evaluating each dimension in the contract, offering a score from 1 (excellent!) to 0 (terrible!).  The current contract is **uncalibrated**, meaning that all the dimensions are equally important, but it's a starting point for learning which are **actually** imporant based on your preferences.

In [ ]:
pi_scores = client.contracts.score(
    contract=aesop_contract,
    llm_input=prompt,
    llm_output=response,
)

for dimension_name, dimension_scores in pi_scores.dimension_scores.items():
  print(f"{dimension_name}: {dimension_scores.total_score}")
  for subdimension_name, subdimension_score in dimension_scores.subdimension_scores.items():
    print(f"\t{subdimension_name}: {subdimension_score}")
  print("\n")
print("---------------------")
print(f"Total score: {pi_scores.total_score}")

## Save it!

Finally, save the Contract so you can come back to it later.

A contract is a simple Pydantic model, which can be serialized to JSON and stored locally.

The cell below will offer a download of the contract.



In [ ]:
from pathlib import Path
from google.colab import files

filename = 'aesop_ai.json'
Path(filename).write_text(aesop_contract.model_dump_json(indent=2))
files.download(filename)

## Next Steps

Now that you have a basic scorer, you want an input set to play with. Proceed to [InputSet Generation](https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/InputSet_Generation.ipynb)